In [ ]:
# Piece detection result table
import json

data = json.loads(open('results.json', 'r').read())

best_mAP = 0
best_mAP_model = None
for ext in ['.pt', '.yaml']:
    for size in ['n', 's', 'm', 'l']:
        row = data['pieces'][f'{size}_det{ext}']
        # "mAP50-95": 0.5855185251296204,
        #    "mAP50-95_var": 0.00013804764431909228,
        #    "mP": 0.6997304885966344,
        #    "mP_var": 0.0002543603331895298,
        #    "mR": 0.7735520081163025,
        #    "mR_var": 0.00012055408945112438,
        print(f"YOLO11 & {size} & {'Yes' if ext == '.pt' else 'No'} & \({row['mP']:.3f} \pm {row['mP_var']**0.5:.1g}\) & \({row['mR']:.3f} \pm {row['mR_var']**0.5:.1g}\) & \({row['mAP50-95']:.3f} \pm {row['mAP50-95_var']**0.5:.1g}\) \\\\")

        if row['mAP50-95'] > best_mAP:
            best_mAP = row['mAP50-95']
            best_mAP_model = f"YOLO11-{size}{ext}"

print(f"\nBest mAP: {best_mAP} for model {best_mAP_model}")


In [ ]:
# Board detection result table
import json

data = json.loads(open('results.json', 'r').read())

best_iou_transform = 0
best_iou_transform_model = None
for ext in ['.pt', '.yaml']:
    for size in ['n', 's', 'm', 'l']:
        # "m_seg.yaml": {
        #    "iou": 0.960867026666729,
        #    "iou_transform": 0.964202948229903,
        #    "best_fold": 2,
        #    "iou_var": 5.450229991418944e-05,
        #    "iou_transform_var": 9.163753355129439e-06

        print(f"YOLO11 & {size} & {'Yes' if ext == '.pt' else 'No'}", end='')
        #\({row['mP']:.3f} \pm {row['mP_var']**0.5:.1g}\) & \({row['mR']:.3f} \pm {row['mR_var']**0.5:.1g}\) & \({row['mAP50-95']:.3f} \pm {row['mAP50-95_var']**0.5:.1g}\) \\\\")

        
        for type in ['pose', 'seg']:
            row = data['board'][f'{size}_{type}{ext}']
            print(f" & \\({row['iou']:.3f} \\pm {row['iou_var']**0.5:.1g}\\) & \\({row['iou_transform']:.3f} \\pm {row['iou_transform_var']**0.5:.1g}\\)", end='')
            if row['iou_transform'] > best_iou_transform:
                best_iou_transform = row['iou_transform']
                best_iou_transform_model = f"YOLO11_{type}-{size}{ext}"

        print(" \\\\")
        
print(f"\nBest iou_trasform: {best_iou_transform} for model {best_iou_transform_model}")


In [ ]:
# import test_json file
import json
test_json = json.loads(open('test_stats.json', 'r').read())

classes = {
    'B': 'Bishop',
    'K': 'King',
    'N': 'Knight',
    'P': 'Pawn',
    'Q': 'Queen',
    'R': 'Rook',
}

average_prec = [0, 0, 0, 0]
average_rec = [0, 0, 0, 0]
for c in classes:
    row = classes[c] 
    for t in ['pred', 'real']:
        for color in [1, 0]:
            key = f"{c.upper() if color == 1 else c.lower()}_{t}"
            correct, wrong = test_json[key]
            row += f" & {correct/(correct + wrong)*100:.2f} \\%"
            if t == 'pred':
                average_prec[2*color] += correct
                average_prec[2*color + 1] += wrong
            else:
                average_rec[2*color] += correct
                average_rec[2*color + 1] += wrong
    print(row + " \\\\")

overall_prec = (average_prec[0] + average_prec[2]) / (average_prec[0] + average_prec[1] + average_prec[2] + average_prec[3])
overall_rec = (average_rec[0] + average_rec[2]) / (average_rec[0] + average_rec[1] + average_rec[2] + average_rec[3])

average_prec = [average_prec[0]/(average_prec[0] + average_prec[1]),
                average_prec[2]/(average_prec[2] + average_prec[3])]
average_rec = [average_rec[0]/(average_rec[0] + average_rec[1]),
                average_rec[2]/(average_rec[2] + average_rec[3])]
print(f"All & {average_prec[1]*100:.2f} \\% & {average_prec[0]*100:.2f} \\% & {average_rec[1]*100:.2f} \\% & {average_rec[0]*100:.2f} \\% \\\\")

print(f"Average precision: {overall_prec*100:.2f}")
print(f"Average recall: {overall_rec*100:.2f}")

print(test_json['pieces_pred'][0]/(test_json['pieces_pred'][0] + test_json['pieces_pred'][1]) * 100)
print(test_json['pieces_real'][0]/(test_json['pieces_real'][0] + test_json['pieces_real'][1]) * 100)



In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

pieces_model_weights = 'runs/pieces/yolo11m_k2_pt/weights/best.pt'
board_model_weights = 'runs/board/yolo11n-seg_k0_pt/weights/best.pt'

TEST_FOLDER_IMAGES = 'dataset/test/images'
TEST_FOLDER_LABELS = 'dataset/test/correct_output'

pieces_model = YOLO(pieces_model_weights)
board_model = YOLO(board_model_weights)

board_to_test = '2475'

pieces_model.predict(f'dataset/test/images/{board_to_test}.png', conf=0.5, save=True)
board_results = board_model.predict(f'dataset/test/images/{board_to_test}.png', conf=0.5, save=True)


mask_contours = board_results[0].masks.xy[0]

window_size = 100
# Convert to numpy array for OpenCV operations
contour_points = np.array(mask_contours, dtype=np.float32)
#contour_points = np.vstack([contour_points, contour_points[0:window_size//10]])  # Close the contour by repeating the first points

# Add 3 points between each adjacent contour_points
new_points = []
for idx in range(len(contour_points)):
    p1 = contour_points[idx]
    p2 = contour_points[(idx + 1) % len(contour_points)]
    # Calculate the distance between p1 and p2
    dist = np.linalg.norm(p2 - p1)
    # Choose number of points based on desired density (e.g., every ~1 pixel)
    num_points = min(int(dist)+1, 100)
    new_points.append(p1)
    for t in [i / num_points for i in range(1, num_points)]:
        interp = p1 + t * (p2 - p1)
        new_points.append(interp)
#contour_points = np.array(new_points, dtype=np.float32)


smoothed_points = []
for i in range(0, len(contour_points), window_size//10):
    start = max(0, i - window_size // 2)
    end = min(len(contour_points), i + window_size // 2 + 1)
    window = contour_points[start:end]
    smoothed_points.append(np.mean(window, axis=0))
#contour_points = np.array(smoothed_points, dtype=np.float32)

# Approximate the contour to a quadrilateral using masks.xy
epsilon = 0.05 * cv2.arcLength(contour_points, True)
board_vert = cv2.approxPolyDP(contour_points, epsilon, True)
rep = 1
while len(board_vert) != 4:
    if len(board_vert) > 4:
        epsilon *= 1.05
    else:
        epsilon *= 0.95
    board_vert = cv2.approxPolyDP(contour_points, epsilon, True)
    rep += 1

    if rep > 50:
        print('Too many iterations, breaking out')
        break
print(f'Found {len(board_vert)} vertices after {rep} iterations')
board_vert = board_vert.reshape(-1, 2)  # Flatten to 2D array
board_vert = [[float(x), float(y)] for x, y in board_vert]




# Calculate distances between adjacent contour points
distances = np.linalg.norm(contour_points - np.roll(contour_points, -1, axis=0), axis=1)
print("Distances between adjacent contour points:", distances)
print("sorted distances:", np.sort(distances))
# Calculate the average distance
average_distance = np.mean(distances)
print("Average distance between adjacent contour points:", average_distance)

# remove points that are too far from the average distance
threshold = average_distance * 5
filtered_points = contour_points[distances < threshold]

# save the image with the detected board with contours in cv2
board_image = cv2.imread(f'dataset/test/images/{board_to_test}.png')
cv2.polylines(board_image, [np.array(board_vert, dtype=np.int32)], True, (0, 255, 0), 2)
cv2.polylines(board_image, [contour_points.astype(np.int32)], True, (255, 0, 0), 2)

#plot the filtered points
for point in contour_points:
    cv2.circle(board_image, tuple(point.astype(int)), 5, (0, 0, 255), -1)
cv2.imwrite('test_board.png', board_image)